## Настройка параметров

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import movie_reviews, stopwords

import warnings
warnings.filterwarnings('ignore')

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:        
        fout.write(str(answer))
        fout.close()

In [3]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [[" ".join(movie_reviews.words(fileids=[f])), 0] for f in negids]
posfeats = [[" ".join(movie_reviews.words(fileids=[f])), 1] for f in posids]

features = negfeats + posfeats

texts = [item[0] for item in features]
target = [item[1] for item in features]

#### 1. Оцените среднее качество ( mean ) и стандартное отклонение ( std ) по fold'ам (cv=5) для:

а) pipeline из CountVectorizer() и LogisticRegression()<br>
б) pipeline из TfidfVectorizer() и LogisticRegression().

В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б


In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

pipeline_count = make_pipeline(CountVectorizer(), LogisticRegression())
pipeline_tfidf = make_pipeline(TfidfVectorizer(), LogisticRegression())

cv_score_count = cross_val_score(pipeline_count, texts, target, cv=5)
mean_count, std_count = np.mean(cv_score_count), np.std(cv_score_count)

cv_score_tfidf = cross_val_score(pipeline_tfidf, texts, target, cv=5)
mean_tfidf, std_tfidf = np.mean(cv_score_tfidf), np.std(cv_score_tfidf)

print(f'Count: mean - {mean_count:.3f}, std - {std_count:.3f}; Tf-Idf: mean - {mean_tfidf:.3f}, \
        std - {std_tfidf:.3f}')

Count: mean - 0.842, std - 0.022; Tf-Idf: mean - 0.821,         std - 0.004


In [5]:
write_answer('submission_1.txt', f'{mean_count} {std_count} {mean_tfidf} {std_tfidf}')

#### 2. Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

$max\_df$ is used for removing terms that appear too frequently, also known as "corpus-specific stop words". For example:

- max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
- max_df = 25 means "ignore terms that appear in more than 25 documents".

The default max_df is 1.0, which means "ignore terms that appear in more than 100% of the documents". Thus, the default setting does not ignore any terms.

$min\_df$ is used for removing terms that appear too infrequently. For example:

- min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
- min_df = 5 means "ignore terms that appear in less than 5 documents".

The default min_df is 1, which means "ignore terms that appear in less than 1 document". Thus, the default setting does not ignore any terms.


In [6]:
pipeline_min_10 = make_pipeline(CountVectorizer(min_df=10), LogisticRegression())
pipeline_min_50 = make_pipeline(CountVectorizer(min_df=50), LogisticRegression())

cv_score_10 = np.mean(cross_val_score(pipeline_min_10, texts, target, cv=5))
cv_score_50 = np.mean(cross_val_score(pipeline_min_50, texts, target, cv=5))

print(f'Accuracy on min_df=10: {cv_score_10:.3f}, on min_df=50: {cv_score_50:.3f}')

Accuracy on min_df=10: 0.840, on min_df=50: 0.816


In [7]:
write_answer('submission_2.txt', f'{cv_score_10} {cv_score_50}')

#### 3. Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

pipeline_logit = make_pipeline(CountVectorizer(), LogisticRegression(random_state=42))
pipeline_svc = make_pipeline(CountVectorizer(), LinearSVC(random_state=42))
pipeline_sgd = make_pipeline(CountVectorizer(), SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=5, shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=42, learning_rate='optimal', eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False))


cv_score_logit = np.mean(cross_val_score(pipeline_logit, texts, target, cv=5))
cv_score_svc = np.mean(cross_val_score(pipeline_svc, texts, target, cv=5))
cv_score_sgd = np.mean(cross_val_score(pipeline_sgd, texts, target, cv=5))

print(f'Accuracy on LogisticReg: {cv_score_logit:.3f}, on Linear SVC: {cv_score_svc:.3f}, on SDG clf: {cv_score_sgd:.3f}')

Accuracy on LogisticReg: 0.842, on Linear SVC: 0.833, on SDG clf: 0.783


In [9]:
min_accuracy = min(cv_score_logit, cv_score_svc, cv_score_sgd)
write_answer('submission_3.txt', f'{min_accuracy}')

#### 4. Подготовьте список стоп-слов с помощью <code>nltk.corpus.stopwords.words('english')</code>, посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [10]:
english_stopwords = stopwords.words('english')
english_stopwords[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [11]:
pipeline_nltk = make_pipeline(CountVectorizer(stop_words=english_stopwords), LogisticRegression())
pipeline_sklearn = make_pipeline(CountVectorizer(stop_words='english'), LogisticRegression())

cv_score_nltk = np.mean(cross_val_score(pipeline_nltk, texts, target, cv=5))
cv_score_sklearn = np.mean(cross_val_score(pipeline_sklearn, texts, target, cv=5))

print(f'Accuracy on nltk stopwords: {cv_score_nltk:.3f}, on sklearn stopwords: {cv_score_sklearn:.3f}')

Accuracy on nltk stopwords: 0.840, on sklearn stopwords: 0.837


In [12]:
write_answer('submission_4.txt', f'{cv_score_nltk} {cv_score_sklearn}')

#### 5. Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра <code>ngram_range</code> и параметр <code>analyzer='char_wb'</code>. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [13]:
pipeline_ngram = make_pipeline(CountVectorizer(ngram_range=(1,2)), LogisticRegression())
pipeline_ngram_char = make_pipeline(CountVectorizer(ngram_range=(3,5), analyzer='char_wb'), LogisticRegression())

cv_score_ngram = np.mean(cross_val_score(pipeline_ngram, texts, target, cv=5))
cv_score_ngram_char = np.mean(cross_val_score(pipeline_ngram_char, texts, target, cv=5))

print(f'Accuracy on 1,2 ngrams: {cv_score_ngram:.3f}, on 3,4,5 char ngrams: {cv_score_ngram_char:.3f}')

Accuracy on 1,2 ngrams: 0.855, on 3,4,5 char ngrams: 0.824


In [14]:
write_answer('submission_5.txt', f'{cv_score_ngram} {cv_score_ngram_char}')